In [1]:
# -*- coding: utf-8 -*-
import time
import os
import tensorflow as tf
import numpy as np
%run Bi_LSTM.py
%run Word2Vec.py
import pandas as pd

C:\Users\dongc\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# 여기부터 하면 됨

In [2]:
from tqdm import tqdm_notebook
from eunjeon import Mecab
from konlpy.tag import Okt

In [3]:
mecab = Mecab()
okt = Okt()
W2V = Word2Vec()

In [15]:
df = pd.read_csv('../data/자소서7.csv',index_col=0, engine='python', encoding='utf8')

## 데이터 섞기

In [16]:
df = df.reset_index().drop(columns={'index'})

In [17]:
rows = np.random.choice(df.index.values, len(df), replace=False)
sampled_df = df.loc[rows]

In [18]:
df = sampled_df

In [19]:
dataset = int(len(df) * 0.7)

In [20]:
df.iloc[:dataset].groupby('category3').agg('count')

,category1,category2,jss,label
category3,,,,
경영지원,1523,1523,1523,1523
생산·물류,2475,2475,2475,2475
서버·시스템,392,392,392,392
영업·마케팅,3873,3873,3873,3873
응용프로그래밍,449,449,449,449
인사,285,285,285,285
재무·금융,1344,1344,1344,1344


In [21]:
df.head()

,category1,category2,jss,label,category3
14044,응용프로그래밍,게임기획,[올리브영 내 마음속에 저장]\n\n타 H&B스토어가 아닌 올리브영을 선택한 이유는...,6,응용프로그래밍
7931,사무,판매·매장관리,"[1년 후]\nSPA 파트너로 입사 후, 처음 업무를 시작할 때 배우는 매장 청결관...",0,경영지원
3386,생산품질,안전·품질·검사·관리,[기본에 충실해야 한다]\n\n저는 동아리 및 봉사활동 활동을 할 때에 있어서 리더...,2,생산·물류
12759,물류,구매관리,"[아모레 퍼시픽의 실크로드를 위해]\n입사 후 저는 제가 맡은 업무에서 인정받고, ...",2,생산·물류
11305,영업,영업관리,상대방에 대한 이해를 바탕으로 공감대를 잘 형성한다는 것이 제 장점입니다. 소대장 ...,3,영업·마케팅


In [22]:
data = []
for txt in tqdm_notebook(df.jss):
    data.append(okt.nouns(txt))

In [23]:
labels = df.label

# 워드 임베딩

In [39]:
config = tf.ConfigProto()
tf.reset_default_graph()
W2V = Word2Vec()

In [40]:
dataset = int(len(data)*0.7)

In [41]:
# 70% train, 30% test
train_X = data[:dataset]
train_Y = labels[:dataset]
test_X = data[dataset:]
test_Y = labels[dataset:]

In [42]:
nb_classes = len(df.category3.unique())

In [43]:
train_X_ = W2V.Convert2Vec("../data/word2vec_okt_nouns.model",train_X)
test_X_ = W2V.Convert2Vec("../data/word2vec_okt_nouns.model",test_X)

In [44]:
train_Y_ = W2V.One_hot(train_Y)
test_Y_ = W2V.One_hot(test_Y)

# 파라미터

In [45]:
Batch_size = 32
Total_size = len(train_X)
Vector_size = 300 # 모델의 워드벡터 사이즈
train_seq_length = [len(x) for x in train_X]
test_seq_length = [len(x) for x in test_X]
Maxseq_length = max(max(train_seq_length), max(test_seq_length))
learning_rate = 0.001
lstm_units = 128
num_class = nb_classes
training_epochs = 4

# 그래프 만들기

In [46]:
X = tf.placeholder(tf.float32, shape = [None, Maxseq_length, Vector_size], name = 'X')
Y = tf.placeholder(tf.float32, shape = [None, num_class], name = 'Y')
seq_len = tf.placeholder(tf.int32, shape = [None])
keep_prob = tf.placeholder(tf.float32, shape = None)

In [47]:
BiLSTM = Bi_LSTM(lstm_units, num_class, keep_prob)

In [48]:
with tf.variable_scope("loss", reuse = tf.AUTO_REUSE):
    logits = BiLSTM.logits(X, BiLSTM.W, BiLSTM.b, seq_len)
    loss, optimizer = BiLSTM.model_build(logits, Y, learning_rate)
prediction = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [49]:
total_batch = int(len(train_X) / Batch_size)
test_batch = int(len(test_X) / Batch_size)

In [50]:
modelName = "./Bidirectional_LSTM_3/BiLSTM_model.ckpt"
saver = tf.train.Saver()

train_acc = []
train_loss = []
test_acc = []
test_loss = []

# 세션 런

In [51]:
sess = tf.Session(config=config)
init = tf.global_variables_initializer()
start_time = time.time()
sess.run(init)
train_writer = tf.summary.FileWriter('./Bidirectional_LSTM_3', sess.graph)
merged = BiLSTM.graph_build()

In [54]:
for epoch in range(training_epochs):  

    avg_acc, avg_loss = 0. , 0.
    mask = np.random.permutation(len(train_X_))
    train_X_ = train_X_[mask]
    train_Y_ = train_Y_[mask]

    for step in tqdm_notebook(range(total_batch)):

        train_batch_X = train_X_[step*Batch_size : step*Batch_size+Batch_size]
        train_batch_Y = train_Y_[step*Batch_size : step*Batch_size+Batch_size]
        batch_seq_length = train_seq_length[step*Batch_size : step*Batch_size+Batch_size]

        train_batch_X = W2V.Zero_padding(train_batch_X, Batch_size, Maxseq_length, Vector_size)

        sess.run(optimizer, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
        # Compute average loss
        loss_ = sess.run(loss, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length,
                                          keep_prob : 0.75})
        avg_loss += loss_ / total_batch

        acc = sess.run(accuracy , feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length,
                                             keep_prob : 0.75})
        avg_acc += acc / total_batch
        print("epoch : {:02d} step : {:04d} loss = {:.6f} accuracy= {:.6f}".format(epoch+1, step+1, loss_, acc))

    summary = sess.run(merged, feed_dict = {BiLSTM.loss : avg_loss, BiLSTM.acc : avg_acc})       
    train_writer.add_summary(summary, epoch)

    t_avg_acc, t_avg_loss = 0. , 0.
    print(avg_loss, avg_acc)
    print("Test cases, could take few minutes")
    
    for step in tqdm_notebook(range(test_batch)):

        test_batch_X = test_X_[step*Batch_size : step*Batch_size+Batch_size]
        test_batch_Y = test_Y_[step*Batch_size : step*Batch_size+Batch_size]
        batch_seq_length = test_seq_length[step*Batch_size : step*Batch_size+Batch_size]

        test_batch_X = W2V.Zero_padding(test_batch_X, Batch_size, Maxseq_length, Vector_size)

        # Compute average loss
        loss2 = sess.run(loss, feed_dict={X: test_batch_X, Y: test_batch_Y, seq_len: batch_seq_length,
                                          keep_prob : 1.0})
        t_avg_loss += loss2 / test_batch

        t_acc = sess.run(accuracy , feed_dict={X: test_batch_X, Y: test_batch_Y, seq_len: batch_seq_length,
                                               keep_prob : 1.0})
        t_avg_acc += t_acc / test_batch

    print("<Train> Loss = {:.6f} Accuracy = {:.6f}".format(avg_loss, avg_acc))
    print("<Test> Loss = {:.6f} Accuracy = {:.6f}".format(t_avg_loss, t_avg_acc))
    train_loss.append(avg_loss)
    train_acc.append(avg_acc)
    test_loss.append(t_avg_loss)
    test_acc.append(t_avg_acc)

epoch : 05 step : 0001 loss = 0.448818 accuracy= 0.906250
epoch : 05 step : 0002 loss = 0.239373 accuracy= 0.968750
epoch : 05 step : 0003 loss = 0.349640 accuracy= 0.937500
epoch : 05 step : 0004 loss = 0.281838 accuracy= 0.968750
epoch : 05 step : 0005 loss = 0.671867 accuracy= 0.781250
epoch : 05 step : 0006 loss = 0.490136 accuracy= 0.875000
epoch : 05 step : 0007 loss = 0.524950 accuracy= 0.875000
epoch : 05 step : 0008 loss = 0.550493 accuracy= 0.843750
epoch : 05 step : 0009 loss = 0.315222 accuracy= 0.937500
epoch : 05 step : 0010 loss = 0.543034 accuracy= 0.843750
epoch : 05 step : 0011 loss = 0.544982 accuracy= 0.781250
epoch : 05 step : 0012 loss = 0.304158 accuracy= 0.906250
epoch : 05 step : 0013 loss = 0.461321 accuracy= 0.781250
epoch : 05 step : 0014 loss = 0.301683 accuracy= 0.937500
epoch : 05 step : 0015 loss = 0.419736 accuracy= 0.875000
epoch : 05 step : 0016 loss = 0.518554 accuracy= 0.875000
epoch : 05 step : 0017 loss = 0.527335 accuracy= 0.906250
epoch : 05 ste

epoch : 05 step : 0142 loss = 0.413894 accuracy= 0.906250
epoch : 05 step : 0143 loss = 0.481418 accuracy= 0.875000
epoch : 05 step : 0144 loss = 0.522869 accuracy= 0.875000
epoch : 05 step : 0145 loss = 0.385237 accuracy= 0.843750
epoch : 05 step : 0146 loss = 0.422976 accuracy= 0.906250
epoch : 05 step : 0147 loss = 0.403772 accuracy= 0.937500
epoch : 05 step : 0148 loss = 0.718124 accuracy= 0.750000
epoch : 05 step : 0149 loss = 0.357649 accuracy= 0.968750
epoch : 05 step : 0150 loss = 0.489764 accuracy= 0.812500
epoch : 05 step : 0151 loss = 0.493045 accuracy= 0.843750
epoch : 05 step : 0152 loss = 0.464060 accuracy= 0.875000
epoch : 05 step : 0153 loss = 0.552883 accuracy= 0.875000
epoch : 05 step : 0154 loss = 0.343497 accuracy= 0.937500
epoch : 05 step : 0155 loss = 0.538230 accuracy= 0.812500
epoch : 05 step : 0156 loss = 0.266071 accuracy= 0.968750
epoch : 05 step : 0157 loss = 0.308671 accuracy= 0.968750
epoch : 05 step : 0158 loss = 0.662360 accuracy= 0.781250
epoch : 05 ste

epoch : 05 step : 0283 loss = 0.607685 accuracy= 0.812500
epoch : 05 step : 0284 loss = 0.291334 accuracy= 0.968750
epoch : 05 step : 0285 loss = 0.746136 accuracy= 0.781250
epoch : 05 step : 0286 loss = 0.388807 accuracy= 0.906250
epoch : 05 step : 0287 loss = 0.359632 accuracy= 0.906250
epoch : 05 step : 0288 loss = 0.444398 accuracy= 0.875000
epoch : 05 step : 0289 loss = 0.751837 accuracy= 0.750000
epoch : 05 step : 0290 loss = 0.212810 accuracy= 0.968750
epoch : 05 step : 0291 loss = 0.547731 accuracy= 0.750000
epoch : 05 step : 0292 loss = 0.431992 accuracy= 0.843750
epoch : 05 step : 0293 loss = 0.289422 accuracy= 0.906250
epoch : 05 step : 0294 loss = 0.634811 accuracy= 0.781250
epoch : 05 step : 0295 loss = 0.495063 accuracy= 0.875000
epoch : 05 step : 0296 loss = 0.438111 accuracy= 0.875000
epoch : 05 step : 0297 loss = 0.412545 accuracy= 0.875000
epoch : 05 step : 0298 loss = 0.488302 accuracy= 0.875000
epoch : 05 step : 0299 loss = 0.335395 accuracy= 0.968750
epoch : 05 ste


<Train> Loss = 0.441906 Accuracy = 0.871324
<Test> Loss = 1.387857 Accuracy = 0.552989


TypeError: cannot concatenate object of type "<class 'numpy.float64'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid

In [ ]:
train_loss = pd.DataFrame({"train_loss":train_loss})
train_acc = pd.DataFrame({"train_acc":train_acc})
test_loss = pd.DataFrame({"test_loss":test_loss})
test_acc = pd.DataFrame({"test_acc":test_acc})
df = pd.concat([train_loss,train_acc,test_loss,test_acc], axis = 1)
df.to_csv("./Bidirectional_LSTM_3/loss_accuracy.csv", sep =",", index=False)

train_writer.close()
duration = time.time() - start_time
minute = int(duration / 60)
second = int(duration) % 60
print("%dminutes %dseconds" % (minute,second))
save_path = saver.save(sess, modelName)

print ('save_path',save_path)